# Response Generation

This notebook sets up embeddings, loads the index, defines the LLM prompt, and runs the retrieval + response pipeline.

## 1. Imports

In [6]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.prompts import PromptTemplate
import pickle


---

## 2. Load embeddings and retriever

In [7]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./multivector_chroma_db_001"
)

with open("parent_documents.pkl", "rb") as f:
    parent_documents = pickle.load(f)


# 🔹 Reconstruís el store
store = InMemoryStore()
store.mset([(d.metadata["id"], d) for d in parent_documents])

# 🔹 Reconstruís el retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key="parent_id",
    search_kwargs={"k": 3} # reminder: top 3 similitud para los chunks resumidos, no para los fallos completos
)

---

## 3. Retrieve documents

In [8]:
# Define LLM
llm = ChatOpenAI(model='o4-mini')

prompt_template = PromptTemplate.from_template("""
Sos un asistente jurídico.
Usá exclusivamente el contenido de los documentos proporcionados para responder la consulta.
Si encontrás documentos relacionados, proporcioná la información correspondiente de dichos documentos.
No inventes ni infieras información que no esté presente en los textos.
Si no hay jurisprudencia relevante, indicá claramente que no la encontrás.
Tampoco menciones ningún documento si no encontraste juridisprudencia relevante.

Documentos:
{context}

Pregunta:
{question}

Respuesta:
""")

combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=prompt_template)

input_text = "¿Existe algún fallo que trate la inconstitucionalidad de la ley de prenda?"

# Step 1: retrieve relevant documents
retrieved_docs = retriever.invoke(input_text)

## 4. Filter irrelevant documents

In [ ]:
from langchain.chains import LLMChain

relevance_prompt = PromptTemplate.from_template("""
¿Aporta este texto información relevante para responder la pregunta siguiente?
Pregunta:
{question}

Texto:
{content}

Responde únicamente “Sí” o “No”.""")
# Step 2: filter irrelevant documents
relevance_chain = LLMChain(llm=llm, prompt=relevance_prompt)

/var/folders/pg/nt5xcr_n6rd6qh7kjj9bnkpr0000gp/T/ipykernel_46343/1521373357.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  relevance_chain = LLMChain(llm=llm, prompt=relevance_prompt)


In [20]:
filtered_docs = []
for doc in retrieved_docs:
    verdict = relevance_chain.run(question=input_text, content=doc.page_content)
    if verdict.strip().lower().startswith("sí"):
        filtered_docs.append(doc)

/var/folders/pg/nt5xcr_n6rd6qh7kjj9bnkpr0000gp/T/ipykernel_46343/1178252960.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  verdict = relevance_chain.run(question=input_text, content=doc.page_content)


## 5. Analyze filtered documents + LLM Final Response

In [ ]:
# Step 3: chain to analyze filtered documents with the question
response = combine_docs_chain.invoke({
    "question": input_text,
    "context": filtered_docs
})

print("\n🧠 Respuesta del modelo:")
print(response)

print("\n📄 Documentos relevantes encontrados:")
for i, doc in enumerate(filtered_docs):
    print(f"\n--- Documento {i+1} ---")
    print("Tribunal:", doc.metadata['Tribunal'])
    print("Sala:", doc.metadata['Sala'])
    print("Expediente:", doc.metadata['Expediente'])
    print("Caratula:", doc.metadata['Caratula'])
    print("Fecha de Sentencia:", doc.metadata['FechaSentencia'])



🧠 Respuesta del modelo:
Sí. En la causa “HSBC Bank Argentina S.A. c/ Martínez, Ramón Vicente s/secuestro prendario” (Fallos 342:1004), la Corte Suprema declaró que el régimen del art. 39 de la Ley 12.962 (prenda con registro) resulta incompatible con los derechos garantizados al consumidor por el art. 42 de la Constitución Nacional y las normas de la Ley 24.240, y rechazó su prelación sobre el estatuto del consumidor.  
  
Esa misma doctrina fue luego aplicada por la Cámara Comercial – Sala F en “HSBC Bank Argentina S.A. c/ García, Dora Claudia s/secuestro prendario” (Expte. COM 5454/2015) y recogida recientemente en “FCA Compañía Financiera S.A. c/ Yglesias Rodrigues, Carlos Rubem s/ secuestro prendario” (Expte. COM 247/2024), donde se declaró inadmisible el secuestro directo sin previa tutela del consumidor, por resultar la norma de la prenda registral “incompatible con la vigencia de varias disposiciones que rigen la defensa del derecho de consumidor”.

📄 Documentos relevantes enco